<a href="https://colab.research.google.com/github/abhishekCS0024/Types-of-RAG/blob/main/LangChain_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain_core langchain_groq langchain_community langchain langgraph

In [ ]:
from langchain_groq import ChatGroq
from google.colab import userdata

In [ ]:
llm=ChatGroq(
    temperature=0,
    groq_api_key=userdata.get('Groq'),
    model_name="qwen/qwen3-32b",
    verbose=True
)

In [ ]:
llm.invoke("hi").content

'<think>\nOkay, the user said "hi". That\'s a greeting. I should respond in a friendly and welcoming way. Maybe ask how I can assist them. Keep it open-ended so they can specify what they need help with. Make sure the tone is positive and approachable. Let me put that together.\n</think>\n\nHello! How can I assist you today? 😊'

In [ ]:
! pip install langchain langchain-community langchain-huggingface chromadb beautifulsoup4 tiktoken sentence-transformers

In [ ]:
# from langchain_community.text_splitters import RecursiveCharacterTextSplitter

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2" # Defining EMBEDDING_MODEL

embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL
)

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

# Add to vectorDB
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=embeddings,
)
retriever = vectorstore.as_retriever()

In [ ]:
! pip install langchain-classic

In [ ]:
# New import path
from langchain_classic import hub

# Pull a prompt
prompt = hub.pull("hwchase17/react")

In [ ]:
# from langchain import hub
from langchain_core.output_parsers import StrOutputParser

# Prompt
prompt = hub.pull("rlm/rag-prompt")

print(f"---PROMPT--- {prompt}")

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Chain
rag_chain = prompt | llm | StrOutputParser()

---PROMPT--- input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [ ]:
# Run
question = "tell me about agent memory."
retrieved_docs = retriever.invoke(question)
formatted_context = format_docs(retrieved_docs)
generation = rag_chain.invoke({"context": formatted_context, "question": question})
print(generation)

<think>
Okay, the user is asking about agent memory. Let me look through the provided context.

First, the context mentions "Memory stream: is a long-term memory module (external database) that records a comprehensive list of agents’ experience in natural language." So that's a key point. Then, in the Agent System Overview, Memory is listed as a component alongside Planning. The system uses LLM as the core, with memory helping the agent behave based on past experiences. Also, there's mention of reflection and refinement, which ties into how memory helps the agent learn from past actions. 

I need to make sure I don't include irrelevant info. The part about chemical weapons and rejected requests might not be directly related to agent memory, so I'll skip that. Focus on the memory stream, its role as an external database, and how it's used with planning and reflection. Keep it concise, three sentences max. Let me check the example answer structure. They used quotes and then explained. Ma